In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [3]:
model = sm.Unet('efficientnetb2', classes=1, activation='sigmoid')
model.load_weights('models/data_l_100e_b2.h5')

2024-02-13 10:29:15.269967: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-02-13 10:29:15.269990: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-02-13 10:29:15.269996: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-02-13 10:29:15.270027: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-13 10:29:15.270042: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# FIXME: reuse from train

def process_image(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def plot_ds_element(background, overlay):
    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(background)
    axs[1].imshow(overlay)
    plt.show()

In [5]:
def mask2rle(img):
    pixels= img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [8]:
data_dir = 'data/test_v2/'

files_ds = tf.data.Dataset.list_files(data_dir + '*.jpg', shuffle=False)
image_ds = files_ds.map(process_image)

ds = tf.data.Dataset.zip((files_ds, image_ds))
ds = ds.batch(30)

result_dict = {}

for (file, image) in ds:
    p = model.predict(image)

    for i in range(len(p)):
        # plot_ds_element(image[i], p[i])
        encoded = mask2rle(np.round(p[i]))

        f = file[i].numpy().decode('utf-8').split('/')[-1] # FIXME: write to file
        result_dict[f] = encoded
        
df = pd.DataFrame(list(result_dict.items()), columns=['ImageId', 'EncodedPixels'])
df.to_csv('submission.csv', index=False)

1/1 [==============================] - 2s 2s/step
